# This notebook will be used for the development of the Capstone Project
### *By Isaac Bautista*

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Week 3 - Segmenting and Clustering Neighborhoods in Toronto

Retrieving the Wikipedia HTML document as plain text, extracting the Toronto table and finding all of its rows in HTML format

In [3]:
result = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
# result.text

html = result.text

soup = BeautifulSoup(html, 'lxml')

tables = soup.find_all('table')
toronto_table = tables[0]

toronto_table_trs = toronto_table.find_all('tr')
toronto_table_trs[0:5]

[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>, <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>]

Cleaning the rows from unwanted characters to obtain an array of string rows

In [4]:
toronto_table_rows = []

toronto_table_ths = toronto_table_trs[0].find_all('th')
toronto_table_ths_str = str(toronto_table_ths)
toronto_table_head = BeautifulSoup(toronto_table_ths_str, 'lxml').get_text()
toronto_table_head = toronto_table_head.lstrip('[')
toronto_table_head = toronto_table_head.rstrip('\n]')
toronto_table_head = ','.join(toronto_table_head.split(', '))
toronto_table_rows.append(toronto_table_head)

for n_toronto_table_tr in toronto_table_trs:
    
    n_toronto_table_tr_tds = n_toronto_table_tr.find_all('td')
    if len(n_toronto_table_tr_tds) == 0:
        continue
    n_toronto_table_tr_tds_str = str(n_toronto_table_tr_tds)
    n_toronto_row = BeautifulSoup(n_toronto_table_tr_tds_str, 'lxml').get_text()
    n_toronto_row = n_toronto_row.lstrip('[')
    n_toronto_row = n_toronto_row.rstrip('\n]')
    n_toronto_row = ','.join(n_toronto_row.split(', '))
    toronto_table_rows.append(n_toronto_row)

toronto_table_rows[0:5]

['Postcode,Borough,Neighbourhood',
 'M1A,Not assigned,Not assigned',
 'M2A,Not assigned,Not assigned',
 'M3A,North York,Parkwoods',
 'M4A,North York,Victoria Village']

Creating the data frame

In [5]:
toronto_df01 = pd.DataFrame(toronto_table_rows)
toronto_df01 = toronto_df01[0].str.split(',', expand=True)

print('toronto_df01.shape:', toronto_df01.shape)
toronto_df01.head()

toronto_df01.shape: (289, 3)


,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Moving up the first row as the data frame header

In [6]:
toronto_df02 = toronto_df01.rename(columns=toronto_df01.iloc[0])
toronto_df02 = toronto_df02.drop([0])
toronto_df02.reset_index(drop=True, inplace=True)
toronto_df02.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)
print('toronto_df02.shape:', toronto_df02.shape)
toronto_df02.head()

toronto_df02.shape: (288, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Removing the rows whose Borough value is Not assigned

In [7]:
toronto_df03 = toronto_df02.drop(toronto_df02[toronto_df02['Borough'] == 'Not assigned'].index)
toronto_df03.reset_index(drop=True, inplace=True)

print('toronto_df03.shape:', toronto_df03.shape)
toronto_df03.head()

toronto_df03.shape: (211, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Joining rows' neighbourhoods by grouping them by Postcode and Borough

In [8]:
toronto_df04 = toronto_df03.groupby(['PostalCode', 'Borough'], sort=False)['Neighborhood'].apply(', '.join).reset_index()

print('toronto_df04.shape:', toronto_df04.shape)
toronto_df04.head()

toronto_df04.shape: (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


Assigning the Borough value to the Neighbourhood that is set to Not assigned for each row

In [9]:
toronto_df = toronto_df04
# toronto_df.replace({'Neighbourhood': 'Not assigned'}, toronto_df[toronto_df['Neighbourhood'] == 'Not assigned']['Borough'].values[0], inplace=True)
toronto_df.loc[toronto_df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = toronto_df.loc[toronto_df['Neighborhood'] == 'Not assigned', 'Borough']

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [10]:
print('toronto_df.shape:', toronto_df.shape)

toronto_df.shape: (103, 3)


Retrieving Toronto geospatial data

In [11]:
toronto_geospatial_data_df = pd.read_csv('https://cocl.us/Geospatial_data')
print('toronto_geospatial_data_df.shape:', toronto_geospatial_data_df.shape)
toronto_geospatial_data_df.head()

toronto_geospatial_data_df.shape: (103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the Toronto a Toronto geospatial dataframes

In [12]:
toronto_df = toronto_df.merge(toronto_geospatial_data_df, left_on='PostalCode', right_on='Postal Code')
toronto_df

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
7,M3B,North York,Don Mills North,M3B,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",M4B,43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",M5B,43.657162,-79.378937
